## **Business Context**
You are provided an e-commerce data set from a real-world organisation to perform customer segmentation with clustering models to improve marketing efforts (SAS, 2024). It is a transnational data set with customers from five continents (Oceania, North America, Europe, Africa, and Asia) and 47 countries.

The data set contains 951,668 rows, each representing a product a customer ordered. The data set contains details about the customer (e.g. location, product type, loyalty member) and order (e.g. days to delivery, delivery date, order date, cost, quantity ordered, profit) based on orders between 1 January 2012 and 30 December 2016.

Your task is to develop a robust customer segmentation to assist the e-commerce company in understanding and serving its customers better. This will help to have a more customer-centric focus, improving marketing efficiency. Therefore, you’ll explore the data, employ preprocessing and feature engineering, dimension reduction, and perform customer segmentation with clustering models.


## **Objective**
Apply statistical and ML methods to perform customer segmentation with clustering techniques.

In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import scipy.cluster.hierarchy as sch
import matplotlib.cm as cm
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.manifold import TSNE

In [ ]:
!pip3 install gdown

In [ ]:
!gdown ''

In [ ]:
df = pd.read_csv('CUSTOMERS_CLEAN.csv')

## Exploratory data analysis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

## Data Preprocessing

In [ ]:
#checking for duplicates
df.duplicated().sum()

In [ ]:
# Removing 21 duplicates in dataframe
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
#checking for null
df.isnull().sum()

In [ ]:
# Replacing null values
df = df.fillna('')
df.isnull().sum().sum()

In [ ]:
len(df['Customer ID'].unique())

In [ ]:
# transforming the data to standardise and for efficient programming
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d%b%Y')
df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'], format='%d%b%Y')
df['Customer_BirthDate'] = pd.to_datetime(df['Customer_BirthDate'], format='%d%b%Y')
df['Total Revenue'] = df['Total Revenue'].replace('[\$,]', '', regex=True).astype(float)
df['Discount']=df['Discount'].replace('[\%]', '', regex=True).replace('   .','0').astype(int)
df['Profit'] = df['Profit'].replace('[\(\)\$,]', '', regex=True).astype(float)
df['Unit Cost'] = df['Unit Cost'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
# checking the status of features after transformations
df.info()

In [ ]:
# the numerical columns are filtered and assigned to dataframe
num_df = df[['Quantity','Total Revenue','Unit Cost','Discount','Profit','Days to Delivery','Loyalty Num']].copy()
num_df

In [ ]:
# visualising numerical columns in boxplots to identify extreme values
plt.figure(figsize=(18, 10))
plt.subplot(3, 3, 1)
plt.boxplot(df['Quantity'])
plt.title('Box Plot of Quantity')

plt.subplot(3, 3, 2)
plt.boxplot(df['Total Revenue'])
plt.title('Box Plot of Total Revenue')

plt.subplot(3, 3, 3)
plt.boxplot(df['Unit Cost'])
plt.title('Box Plot of Unit Cost')

plt.subplot(3, 3, 4)
plt.boxplot(df['Discount'])
plt.title('Box Plot of Discount')

plt.subplot(3, 3, 5)
plt.boxplot(df['Profit'])
plt.title('Box Plot of Profit')

plt.subplot(3, 3, 6)
plt.boxplot(df['Days to Delivery'])
plt.title('Box Plot of Days to Delivery')

plt.subplot(3, 3, 7)
plt.boxplot(df['Loyalty Num'])
plt.title('Box Plot of Loyalty Num')

plt.tight_layout()
plt.show()

Detecting outliers using IQR

In [ ]:
outlier_df = num_df.copy()
for col in outlier_df.columns:

  Q1 = outlier_df[col].quantile(0.25)
  Q3 = outlier_df[col].quantile(0.75)
  IQR = Q3 - Q1

  outlier_df[f'Outlier_{col}'] = ((outlier_df[col] < (Q1 - 1.5 * IQR)) | (outlier_df[col] > (Q3 + 1.5 * IQR))).astype(int)

outlier_df

# Feature Engineering

Creating new features for frequency, recency, CLV, average unit cost, and customer age

In [ ]:
df['Frequency'] = df.groupby('Customer ID')['Order ID'].transform('count')
df['Frequency']

In [ ]:
df['Recency'] = (pd.to_datetime('today') - df['Delivery_Date']).dt.days
df['Recency']

In [ ]:
df['CLV'] = df.groupby('Customer ID')['Total Revenue'].transform('sum')
df['CLV']

In [ ]:
df['Avg_UnitCost'] = df.groupby('Customer ID')['Unit Cost'].transform('mean')
df['Avg_UnitCost']

In [ ]:
today = pd.Timestamp('now')
df['customer_age'] = (today - df['Customer_BirthDate']).dt.days // 365
df['customer_age']

## Data Transformations

In [ ]:
# aggregation operations and groupy() function was used to get one customer per row.
agg_operations = {
    'Frequency':'max',
    'Recency':'min',
    'CLV':'max',
    'customer_age':'max',
    'Avg_UnitCost':'mean'
}

df = df.groupby('Customer ID').agg(agg_operations).reset_index()
df_aggr = df[['Frequency','Recency','CLV','customer_age','Avg_UnitCost']]

## Clustering

For more efficient programming, incorporated the pipeline method to use column transformers and k-means clustering.

In [ ]:
preprocessor = ColumnTransformer(
  transformers=[('num', StandardScaler(), ['Frequency','Recency','CLV','customer_age','Avg_UnitCost'])]
)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=10, n_init=10)
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('kmeans', kmeans)])
pipeline.fit(df_aggr)

Getting preprocessed data from pipeline

In [ ]:
df_preprocessed = pipeline.named_steps['preprocessor'].transform(df_aggr)

Dimensionality reduction using PCA

In [ ]:
pca = PCA(n_components=2)
pca_df = pd.DataFrame(pca.fit_transform(df_preprocessed), columns=['pca_1', 'pca_2'])

Elbow method

In [ ]:
wcss = []

for k in range(2, 16):
  pipeline.named_steps['kmeans'].set_params(n_clusters=k)
  pipeline.fit(df_aggr)
  wcss.append(pipeline.named_steps['kmeans'].inertia_)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(wcss, marker='o')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.xticks(range(0, len(wcss)), range(2, len(wcss) + 2))
plt.grid(True)
plt.show()

Silhouette analysis

In [ ]:
silhouette_avgs = []
fig, axes = plt.subplots(5, 2, figsize=(16, 24))
fig.suptitle("KMeans Clustering Analysis", fontsize=14, fontweight='bold')

for k in range(2, 7):
  ax1, ax2 = axes[k-2]
  pipeline.named_steps['kmeans'].set_params(n_clusters=k)
  pipeline.fit(df_aggr)

  labels = pipeline.named_steps['kmeans'].labels_
  silhouette_avg = silhouette_score(df_preprocessed, labels)
  silhouette_avgs.append(silhouette_avg)
  sample_silhouette_values = silhouette_samples(df_preprocessed, labels)

  y_lower = 10
  for i in range(k):
    ith_cluster_silhouette_values = sample_silhouette_values[labels == i]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / k)
    ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    y_lower = y_upper + 10

  ax1.set_title(f"The silhouette plot for the various clusters (n_clusters = {k}).")
  ax1.set_xlabel("The silhouette coefficient values")
  ax1.set_ylabel("Cluster label")
  ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
  ax1.set_yticks([])
  ax1.set_xticks(np.arange(-0.1, 1.1, 0.2))

  colors = cm.nipy_spectral(labels.astype(float) / k)
  ax2.scatter(pca_df['pca_1'], pca_df['pca_2'], marker='.', s=30, lw=0, alpha=0.7, c=colors, edgecolor='k')
  centers = pca.transform(pipeline.named_steps['kmeans'].cluster_centers_)
  ax2.scatter(centers[:, 0], centers[:, 1], marker='o', c="white", alpha=1, s=200, edgecolor='k')
  for i, c in enumerate(centers):
    ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')
  ax2.set_title("The visualization of the clustered data.")
  ax2.set_xlabel("Feature space for the 1st PCA component")
  ax2.set_ylabel("Feature space for the 2nd PCA component")

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(silhouette_avgs, marker='o')
plt.title('Silhouette Score vs. Number of Clusters')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Average Silhouette Score')
plt.xticks(range(0, len(silhouette_avgs)), range(2, len(silhouette_avgs) + 2))
plt.grid(True)
plt.show()

In [ ]:
# from the above visualisation we found out the best K as 5
k=5
pipeline.named_steps['kmeans'].set_params(n_clusters=k)
pipeline.fit(df_aggr)
labels = pipeline.named_steps['kmeans'].labels_
labels

In [ ]:
df['cluster'] = labels
df

PCA for visualisation

In [ ]:
pca_df['cluster'] = labels
pca_df

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(
  x='pca_1',
  y='pca_2',
  hue='cluster',
  data=pca_df,
  palette='Set1',
  marker='.',
  s=30,
  linewidth=0,
  alpha=0.7
)
centers = pca.transform(pipeline.named_steps['kmeans'].cluster_centers_)
sns.scatterplot(x=centers[:, 0], y=centers[:, 1], color="white", alpha=1, s=200, edgecolor='k', legend=False)
for i, c in enumerate(centers):
  plt.scatter(x=c[0], y=c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')
plt.title('PCA Plot with Cluster Labels')
plt.xlabel("Feature space for the 1st PCA component")
plt.ylabel("Feature space for the 2nd PCA component")
plt.show()

t-SNE for visualisation

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(df_preprocessed)
df_tsne = pd.DataFrame(data=tsne_result, columns=['t-SNE1', 't-SNE2'])


In [ ]:
df_tsne['cluster'] = labels
df_tsne

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
  x='t-SNE1',
  y='t-SNE2',
  hue='cluster',
  data=df_tsne,
  palette='Set1',
  marker='.',
  s=30,
  linewidth=0,
  alpha=0.7
)
plt.title('t-SNE Visualization of Clusters')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

BoxPlot Visualisation (Features Vs Clusters)

In [ ]:
for column in df_aggr[['Frequency','Recency','CLV','customer_age','Avg_UnitCost']]:
  plt.figure(figsize=(12, 6))
  sns.boxplot(
    x='cluster',
    y=column,
    data=df,
    hue='cluster',
    legend=False,
    palette='Set1'
  )
  plt.title(f'Boxplot of {column} by Cluster')
  plt.xlabel('Cluster')
  plt.ylabel(column)
  plt.show()

Agglomerative Clustering

Since the notebook was crashing due to huge amout of data, data sampled to 10000 rows to fix the issue.

In [ ]:
small_df = df_aggr.sample(n= 10000).reset_index(drop = True)

In [ ]:
agglo_cluster = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage='average')
small_df['cluster'] = agglo_cluster.fit_predict(small_df)
small_df

In [ ]:
Z = linkage(small_df, method='average')
plt.figure(figsize=(15, 10))
dendrogram(Z)
plt.title('Dendrogram for Customer Data')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.show()


> The objective was to develop a robust customer segmentation to assist the e-commerce company in understanding and serving its customers better. The dataset had 20 features to choose on which features or feature combinations can help in customer segmentation. The steps followed are exploratory data analysis, data preprocessing, feature engineering and determining the best number of clusters(k) using the Elbow and Silhouette score methods and based on optimum number of ‘k’, performing the k-means clustering. The resulting clusters can then be visualised in 2D by reducing the dimensions using PCA and t-SNE models.


### Reference:
SAS, 2024. CUSTOMERS_CLEAN [Data set]. SAS. Last revised on 15 December 2021. [Accessed 20 February 2024].